Vamos começar com a análise exploratória do dataset Favorita(a famosa EDA - Exploratory Data Analysis). 

Este notebook vai conter:

Dados Utilizados (do Kaggle)

    - train.csv -- vendas por loja/produto/dia
    - stores.csv -- dados das lojas
    - items.csv -- dados dos produtos
    - oil.csv -- preço do petróleo(usado como variável macroeconômica)
    - holidays_events.csv -- feriados e eventos
    - transactions -- número de transações por loja/dia
    - test.csc -- dados para prever